Train model

In [ ]:
# Cài đặt thư viện
!pip install pyvi

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import chardet
import re
import string
import pickle
import matplotlib.pyplot as plt
from pyvi import ViTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Bidirectional, Dense, BatchNormalization

# Mount Google Drive
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')

# Tải lên tệp văn bản
uploaded = files.upload()

# Chuẩn bị dữ liệu
data = ""
for filename in uploaded.keys():
    with open(filename, "rb") as f:
        encoding = chardet.detect(f.read())['encoding']
    with open(filename, "r", encoding=encoding) as f:
        data += f.read()

corpus = data.lower().split("\n")

# Lưu kho văn bản vào tập tin
corpus_filename = 'corpus.txt'
with open(corpus_filename, 'w') as f:
    f.write('\n'.join(corpus))

!cp corpus.txt "/content/drive/My Drive/"

# Xác định một hàm để tiền xử lý dữ liệu văn bản.
def preprocess_text(text):
    # Loại bỏ đường dẫn URL, tên người dùng và các từ khóa trong hashtag
    text = re.sub(r'(https?:\/\/[^\s]*)|(www\.[^\s]*)|(@[^\s]*)|(#\w+)', '', text)
    # Chuyển đổi chữ hoa thành chữ thường trong văn bản
    text = text.lower()
    # Ghép từ tiếng Việt
    text = ViTokenizer.tokenize(text)
    # Xóa dấu câu và ký tự thừa
    text = text.translate(str.maketrans('', '', string.punctuation.replace("_", "")))
    # Trả lại văn bản được xử lý trước
    return text

# Tiền xử lý dữ liệu văn bản và mã hóa các từ
preprocessed_lines = [preprocess_text(line) for line in corpus]

# Chia dữ liệu thành 70% train và 30% test
train_data, test_data = train_test_split(preprocessed_lines, test_size=0.3, random_state=42)

# Tiền xử lý dữ liệu cho tập train
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in train_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

predictors_train, label_train = input_sequences[:, :-1], input_sequences[:, -1]
label_train = tf.keras.utils.to_categorical(label_train, num_classes=total_words)

# Tiền xử lý dữ liệu cho tập test
test_sequences = []
for line in test_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        test_sequences.append(n_gram_sequence)

test_sequences = np.array(pad_sequences(test_sequences, maxlen=max_sequence_len, padding='pre'))

predictors_test, label_test = test_sequences[:, :-1], test_sequences[:, -1]
label_test = tf.keras.utils.to_categorical(label_test, num_classes=total_words)

# Tạo dataset cho việc train và test mô hình
train_dataset = tf.data.Dataset.from_tensor_slices((predictors_train, label_train)).shuffle(len(predictors_train)).batch(128)
test_dataset = tf.data.Dataset.from_tensor_slices((predictors_test, label_test)).batch(128)

# Xây dựng mô hình LSTM
model = tf.keras.Sequential([
    Embedding(total_words, 100),
    Bidirectional(LSTM(150, return_sequences=True)),
    BatchNormalization(),
    LSTM(100),
    Dropout(0.2),
    Dense(total_words, activation='softmax')
])

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Sử dụng EarlyStopping callback để dừng quá trình train sớm hơn nếu không có sự cải thiện đáng kể về độ chính xác
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Sử dụng ModelCheckpoint callback để lưu lại mô hình tốt nhất trong quá trình train
model_checkpoint = ModelCheckpoint('/content/drive/My Drive/best_model.h5', monitor='val_loss', save_best_only=True)

# Train model
history = model.fit(train_dataset, epochs=100, validation_data=test_dataset, callbacks=[early_stopping, model_checkpoint])

# Đánh giá mô hình trên tập test
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Lưu lại mô hình
model_filename = 'model.h5'
model.save(model_filename)

!cp model.h5 "/content/drive/My Drive/"

with open('/content/drive/My Drive/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

np.save('/content/drive/MyDrive/sequences.npy', input_sequences)

# Vẽ đồ thị biểu diễn sự biến thiên của hàm mất mát trong quá trình huấn luyện và đánh giá
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(history.history['loss'], label='train')
ax.plot(history.history['val_loss'], label='val')
ax.set_xlabel('Epochs')
ax.set_ylabel('Loss')
ax.set_title('Training Loss Curve')
ax.legend()
plt.savefig('/content/drive/MyDrive/train_loss.png')

# Vẽ đồ thị biểu diễn sự biến thiên của độ chính xác trong quá trình huấn luyện và đánh giá
fig, ax = plt.subplots(1, 1, figsize=(10, 6))
ax.plot(history.history['accuracy'], label='train')
ax.plot(history.history['val_accuracy'], label='val')
ax.set_xlabel('Epochs')
ax.set_ylabel('Accuracy')
ax.set_title('Training Accuracy Curve')
ax.legend()
plt.savefig('/content/drive/MyDrive/train_acc.png')

# Vẽ đồ thị kiến trúc mô hình
plot_model(model, to_file='/content/drive/MyDrive/model_structure.png', show_shapes=True, rankdir='TB')

# Đánh giá mô hình trên tập test
test_loss, test_acc = model.evaluate(test_dataset)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

Train tiếp model

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from google.colab import files
from google.colab import drive
import chardet
import re
import string
import pickle
import matplotlib.pyplot as plt
from pyvi import ViTokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout
import os
from tensorflow.keras.utils import plot_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Mount Google Drive
drive.mount('/content/drive')

# Đặt các đường dẫn
root_path = '/content/drive/My Drive/'
model_path = os.path.join(root_path, 'model.h5')
tokenizer_path = os.path.join(root_path, 'tokenizer.pkl')
corpus_path = os.path.join(root_path, 'corpus.txt')

# Tải mô hình và công cụ tokenizer đã được đào tạo trước
model = tf.keras.models.load_model(model_path)
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Tải lên tệp dữ liệu văn bản mới
uploaded = files.upload()

# Thêm dữ liệu mới vào ngữ liệu và tiền xử lý
data = ""
for filename in uploaded.keys():
    with open(filename, "rb") as f:
        encoding = chardet.detect(f.read())['encoding']
    with open(filename, "r", encoding=encoding) as f:
        data += f.read()
new_corpus = data.lower().split("\n")

# Kết hợp dữ liệu cũ và mới
if os.path.exists(corpus_path):
    with open(corpus_path, 'r') as f:
        corpus = f.read().lower().split('\n')
else:
    corpus = []
corpus += new_corpus
corpus = list(filter(None, corpus))

# Xác định một hàm để tiền xử lý dữ liệu văn bản.
def preprocess_text(text):
    # Loại bỏ đường dẫn URL, tên người dùng và các từ khóa trong hashtag
    text = re.sub(r'(https?:\/\/[^\s]*)|(www\.[^\s]*)|(@[^\s]*)|(#\w+)', '', text)
    # Chuyển đổi chữ hoa thành chữ thường trong văn bản
    text = text.lower()
    # Ghép từ tiếng Việt
    text = ViTokenizer.tokenize(text)
    # Xóa dấu câu và ký tự thừa
    text = text.translate(str.maketrans('', '', string.punctuation.replace("_", "")))
    # Trả lại văn bản được xử lý trước
    return text

preprocessed_lines = [preprocess_text(line) for line in new_corpus]

# Chia dữ liệu thành 70% train và 30% test
train_data, test_data = train_test_split(preprocessed_lines, test_size=0.3, random_state=42)

tokenizer.fit_on_texts(new_corpus)
total_words = len(tokenizer.word_index) + 1

# Lấy kích thước của lớp cuối cùng và cập nhật nó
final_layer_shape = model.layers[-1].output_shape
new_output_layer = Dense(total_words, activation='softmax')(model.layers[-2].output)
model.layers[-1] = Dense(total_words, activation='softmax')
model = tf.keras.models.Model(inputs=model.input, outputs=new_output_layer)

# Tiền xử lý dữ liệu cho tập train
input_sequences = []
for line in train_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

xs, labels = input_sequences[:,:-1],input_sequences[:,-1]
ys = to_categorical(labels, num_classes=total_words)
train_data, test_data, train_label, test_label = train_test_split(xs, ys, test_size=0.3, random_state=42)


# predictors_train, label_train = input_sequences[:, :-1], input_sequences[:, -1]

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

label_train = to_categorical(labels)

# Tiền xử lý dữ liệu cho tập test
test_sequences = []
for line in test_data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        test_sequences.append(n_gram_sequence)

max_len = max([len(x) for x in test_sequences])
test_sequences = np.array(pad_sequences(test_sequences, maxlen=max_len, padding='pre'))
# test_sequences = np.array(pad_sequences(test_sequences, maxlen=max_sequence_len, padding='pre'))

test_xs, test_labels = test_sequences[:,:-1],test_sequences[:,-1]
test_ys = to_categorical(test_labels, num_classes=total_words)

# predictors_test, label_test = test_sequences[:, :-1], test_sequences[:, -1]
# label_test = to_categorical(label_test, num_classes=total_words)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

callbacks=[EarlyStopping(patience=5, monitor='val_loss'),
ModelCheckpoint(filepath=model_path, save_best_only=True, monitor='val_loss')]
history = model.fit(train_data, train_label, epochs=100, verbose=1, validation_data=(test_xs, test_ys), callbacks=callbacks)
# history = model.fit(xs, ys, epochs=100, verbose=1, validation_data=(test_xs, test_ys), callbacks=callbacks)

# Define early stopping and model checkpoint callbacks
# early_stopping = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
# model_checkpoint = ModelCheckpoint(os.path.join(root_path, 'best_model.h5'), monitor='val_loss', mode='min', verbose=1, save_best_only=True)

# Retrain the model with early stopping and model checkpoint callbacks
# with tf.device('/device:GPU:0'):
#     history = model.fit(predictors_train, label_train, epochs=100, verbose=1, validation_data=(predictors_test, label_test), callbacks=[early_stopping, model_checkpoint])

# Đánh giá mô hình trên tập test
# test_loss, test_acc = model.evaluate(predictors_test, label_test)
test_loss, test_acc = model.evaluate(test_xs, test_ys)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_acc)

# Lưu mô hình và công cụ tokenizer đã được cập nhật
model.save(model_path)
with open(tokenizer_path, 'wb') as f:
  pickle.dump(tokenizer, f)

# Lưu lại dữ liệu đã được cập nhật
with open(corpus_path, 'w') as f:
  f.write('\n'.join(corpus))

# Vẽ đồ thị biểu diễn sự biến thiên của hàm mất mát trong quá trình huấn luyện
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Lưu đồ thị vào Google Drive
plt.savefig(os.path.join(root_path, 'loss.png'))

# Vẽ đồ thị biểu diễn sự biến thiên của độ chính xác trong quá trình huấn luyện
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Lưu đồ thị vào Google Drive
plt.savefig(os.path.join(root_path, 'accuracy.png'))

# Vẽ đồ thị kiến trúc mô hình
plot_model(model, to_file='/content/drive/MyDrive/model_structure.png', show_shapes=True, rankdir='TB')



Tính điểm BLEU

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle
from google.colab import drive
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import numpy as np
import os

# Mount Google Drive
drive.mount('/content/drive')

# Tải mô hình và công cụ tokenizer được đào tạo trước từ tệp pickle
root_path = '/content/drive/My Drive/'
model_path = os.path.join(root_path, 'model.h5')
tokenizer_path = os.path.join(root_path, 'tokenizer.pkl')

model = tf.keras.models.load_model(model_path)
with open(tokenizer_path, 'rb') as f:
    tokenizer = pickle.load(f)

# Tạo ra văn bản bằng cách sử dụng mô hình đã được đào tạo trước
seed_text = "bài báo"
next_words = 11
generated_text = seed_text
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=model.input_shape[1], padding='pre')
    predicted = np.argmax(model.predict(token_list), axis=-1)
    output_word = ""
    for word, index in tokenizer.word_index.items():
        if index == predicted:
            output_word = word
            break
    seed_text += " " + output_word
    generated_text += " " + output_word

# Đánh giá văn bản được tạo ra bằng cách sử dụng điểm BLEU với kỹ thuật smoothing
reference_text = ["bài báo nêu rõ các vấn đề của nền kinh tế Việt Nam"]

hypothesis_text = generated_text.split()
reference_text = reference_text[0].split()

smoothie = SmoothingFunction().method4
bleu_score = sentence_bleu([reference_text], hypothesis_text, smoothing_function=smoothie)

# In ra văn bản được tạo ra và điểm BLEU tương ứng của nó
print("Generated text:", generated_text)
print("reference text:", ' '.join(reference_text))
print("BLEU score:", bleu_score)